In [1]:
from loader import Loader
from parcellation import Parcellation
from measures.correlation import CorrelationBasedBrainNetwork
from measures.covariance import CovarianceBasedBrainNetwork
from measures.causality import GrangerCausalityBasedBrainNetwork
from d3blocks import D3Blocks

In [2]:
group = 'SCZ'
modality = 'BOLD'
task = 'task001_run001'
kind = 'bold_mcf_brain'

In [4]:
agent = Loader()
data = agent.filter_data(group=group, modality=modality, task=task, kind=kind)
agent = Parcellation(data)
parcellation_results = agent.parcellate()

In [12]:
parcellation_results['sub001'][1].shape

(400, 3)

In [5]:
parcellation_results['sub001'][0]

,Precentral_L,Precentral_R,Frontal_Sup_L,Frontal_Sup_R,Frontal_Sup_Orb_L,Frontal_Sup_Orb_R,Frontal_Mid_L,Frontal_Mid_R,Frontal_Mid_Orb_L,Frontal_Mid_Orb_R,...,Cerebelum_10_L,Cerebelum_10_R,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9,Vermis_10
0,1.610692,2.291672,2.047640,2.428134,2.456172,1.880481,2.632550,2.917690,1.851532,1.657773,...,0.248966,0.362172,0.590110,-0.838515,-1.339095,-0.370157,1.121556,0.357268,0.824682,2.345670
1,0.294464,1.287140,0.816706,0.605587,1.661697,1.236485,0.373880,1.309953,1.025004,0.922508,...,-0.301395,-0.082882,0.458437,-1.129146,-0.967490,-0.436557,0.596637,0.006525,1.555604,1.403181
2,-0.775580,-0.029949,1.683969,1.678228,1.550253,0.882822,1.219480,2.628971,1.643769,0.761141,...,0.811507,-0.747500,-0.282949,-0.221228,-0.570342,-0.055012,1.279994,0.888802,2.135714,1.982446
3,-2.145952,-1.107416,1.653646,2.048225,1.959936,2.263871,0.945507,2.302218,1.672192,2.398243,...,2.087762,1.408344,0.447386,-0.778180,-1.020834,-1.160715,0.411783,0.759826,1.452666,1.984178
4,-2.268650,-1.341042,0.390538,1.610161,1.127515,1.523904,-1.051461,1.372379,1.720981,1.404419,...,1.685889,0.338406,0.000345,-0.195802,-0.587146,-1.298796,0.161469,0.329243,0.989565,1.087525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,-1.058975,0.009665,-1.067523,-0.936444,1.441616,0.892968,-1.031290,-0.193885,1.048340,0.360092,...,0.650530,0.372040,-1.505828,-1.454001,-1.651613,-0.043203,0.249841,0.150994,0.119961,0.612489
133,-0.525604,-0.567742,-0.927656,-0.180007,1.256809,1.567834,-0.804678,0.075041,-0.014319,0.917540,...,0.916953,-0.073014,-1.048812,-2.055344,-1.400604,-0.243995,0.054455,0.305881,-0.676202,0.485880
134,-0.125008,-0.072093,-0.266895,0.075484,1.410989,0.712222,-0.673574,0.264556,0.217664,0.653907,...,0.797245,0.005489,-0.717944,-1.472967,-1.061888,-0.408984,-0.923593,-0.701287,-0.253210,0.377605
135,-0.260582,-0.837030,0.124085,0.111067,0.988386,1.077052,-0.314014,-0.324481,0.836429,1.387516,...,1.063668,0.168441,-0.954740,-0.809182,-0.609920,0.160719,-0.473649,-0.431086,-0.615754,0.461842


In [6]:
subject_data, atlas_coordinates = parcellation_results['sub001']

In [7]:
correlation_matrix = CorrelationBasedBrainNetwork(subject_data, sparsify=True).generate_connectivity_matrix()

In [16]:
from utils import *

In [18]:
a = subject_data.diff().dropna()

In [21]:
l = []

In [22]:
for col in a.columns:
    l.append(acf_look_back_period(a[col], 200))

In [24]:
set(l)

{1, 2, 3, 4}

In [35]:
from statsmodels.tsa.stattools import ccf
import numpy as np

def significant_lag(series1, series2, n_lags):
    """
    Calculate the significant lag between two time series.

    Parameters:
    - series1: array-like, first time series
    - series2: array-like, second time series
    - n_lags: int, number of lags to consider

    Returns:
    - int, the lag with the most significant cross-correlation
    """
    # Calculate the cross-correlation
    cross_corr = ccf(series1, series2, adjusted=False)[:n_lags + 1]

    # Calculate the confidence intervals (assuming normal distribution)
    confidence_interval = 1.96 / np.sqrt(len(series1))
    
    # # Identify the lag where the cross-correlation is significant
    significant_lags = np.where(np.abs(cross_corr) > confidence_interval)[0]
    
    return significant_lags
    # if len(significant_lags) == 0:
    #     return None  # No significant lags found
    # else:
    #     # Return the lag with the maximum absolute cross-correlation value
    #     return significant_lags[np.argmax(np.abs(cross_corr[significant_lags]))]

# Example usage:
series1 = np.random.randn(100)
series2 = np.random.randn(100)
n_lags = 20
result = significant_lag(series1, series2, n_lags)
print(f'The most significant lag is: {result}')


The most significant lag is: []


In [38]:
import numpy as np
from scipy import signal
x = [4,4,4,4,6,8,10,8,6,4,4,4,4,4,4,4,4,4,4,4,4,4,4]
y = [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,6,8,10,8,6,4,4]
correlation = signal.correlate(x-np.mean(x), y - np.mean(y), mode="full")
lags = signal.correlation_lags(len(x), len(y), mode="full")
lag = lags[np.argmax(abs(correlation))]

In [39]:
lag

-12

In [27]:
result

array([ 0.09378969, -0.16495624, -0.04671508,  0.07811189,  0.0282055 ,
       -0.044986  ,  0.05651525,  0.01637561, -0.05916782, -0.0057262 ,
        0.12287752, -0.04758561, -0.05381741,  0.11612945,  0.08596906,
       -0.09811661,  0.08935193,  0.07551193, -0.01361263,  0.01297518,
        0.06904262])

In [30]:
from statsmodels.tsa.stattools import acf, pacf
import numpy as np
from scipy import stats


def acf_look_back_period(series, n_lags):
    acf_, confidence_interval = acf(series, nlags=n_lags, alpha=0.05)
    centered_ci = confidence_interval - acf_[:, None]
    outside = np.abs(acf_) >= centered_ci[:, 1]
    # return np.argmax(~outside)
    return acf_, confidence_interval

In [34]:
acf_look_back_period(series1, 10)

(array([ 1.        , -0.02626127, -0.02167804,  0.03737978,  0.09673725,
        -0.04107087, -0.1386327 , -0.00749   , -0.08881574, -0.17494106,
        -0.06963319]),
 array([[ 1.        ,  1.        ],
        [-0.22225767,  0.16973513],
        [-0.21780957,  0.17445348],
        [-0.15884376,  0.23360332],
        [-0.09975964,  0.29323414],
        [-0.2393888 ,  0.15724707],
        [-0.3372771 ,  0.06001171],
        [-0.20981692,  0.19483691],
        [-0.29115331,  0.11352182],
        [-0.37877074,  0.02888862],
        [-0.27915132,  0.13988494]]))

In [37]:
(acf_look_back_period(series1, 10)[1] - acf_look_back_period(series1, 10)[0][:, None])[:, 1]

array([0.        , 0.1959964 , 0.19611745, 0.19714484, 0.20348822,
       0.21093875, 0.2126866 , 0.21268663, 0.21949694, 0.21952631,
       0.22009036])

In [13]:
graph = AssociationGraph(correlation_matrix, atlas_coordinates)
graph.add_edges_from_matrix()

In [14]:
graph.draw_chord()

<Figure size 1500x1500 with 0 Axes>

In [15]:
import numpy as np

class MatrixManipulator:
    def __init__(self, matrix):
        self.matrix = np.array(matrix)
    
    def separate_positive_negative(self):
        positive_matrix = np.where(self.matrix > 0, self.matrix, 0)
        negative_matrix = np.where(self.matrix < 0, self.matrix, 0)
        return positive_matrix, negative_matrix
    
    def binarize(self, threshold):
        binary_matrix = np.where(self.matrix >= threshold, 1, 0)
        return binary_matrix

Positive Matrix:
 [[1 0 3]
 [0 5 0]
 [7 0 9]]
Negative Matrix:
 [[ 0 -2  0]
 [-4  0 -6]
 [ 0 -8  0]]
Binary Matrix:
 [[0 0 1]
 [0 1 0]
 [1 0 1]]
